In [2]:
import metrics_calc

c:\Users\97254\Desktop\NLP_FINAL_PROJECT_DREAMS\dreams_interpreter\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [3]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Check if GPU (CUDA) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Load trained T5 model and tokenizer
model_path = "./dream_interpreter_t5"
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)  # Move model to GPU if available
tokenizer = T5Tokenizer.from_pretrained(model_path)

Using device: cuda


In [4]:
# Read validation dataset (Excel file)
input_excel = "DREAMS DATA\dreams_and_interpretations_Freud.xlsx"  # Replace with your actual Excel file name
df = pd.read_excel(input_excel)

output_csv = "dreams_with_generated_interpretations_t5.csv"


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\97254\AppData\Local\Temp\ipykernel_23268\3878873843.py:2: SyntaxWarning: invalid escape sequence '\d'
  input_excel = "DREAMS DATA\dreams_and_interpretations_Freud.xlsx"  # Replace with your actual Excel file name


In [5]:
# Ensure the dataset has a 'Dream' column
if 'Dream' not in df.columns:
    raise ValueError("Excel file must contain a 'Dream' column.")

# # Function to generate interpretations
# def generate_interpretation(dream):
#     input_text = f"interpret the dream: {dream}"
    
#     # Tokenize and move input tensors to the correct device (GPU if available)
#     input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

#     # Generate text
#     output_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)
#     interpretation = tokenizer.decode(output_ids[0], skip_special_tokens=True)

#     return interpretation.strip()

def generate_interpretation(dream):
    input_text = f"interpret the dream: {dream}"
    
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Improved decoding strategy
    output_ids = model.generate(
        input_ids, 
        max_length=50, 
        num_return_sequences=1, 
        temperature=0.7,  # Controls randomness (lower = more deterministic)
        top_k=50,         # Filters unlikely words (smaller = safer)
        top_p=0.92,       # Nucleus sampling
        repetition_penalty=1.2,  # Penalizes repetition
        no_repeat_ngram_size=2   # Avoids repeating 2-grams
    )
    
    interpretation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return interpretation.strip()


# Generate interpretations
df["Generated_Interpretation"] = df["Dream"].apply(generate_interpretation)

# Save to new CSV
df.to_csv(output_csv, index=False)

print(f"Generated interpretations saved to {output_csv}")


c:\Users\97254\Desktop\NLP_FINAL_PROJECT_DREAMS\dreams_interpreter\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\97254\Desktop\NLP_FINAL_PROJECT_DREAMS\dreams_interpreter\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.92` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


KeyboardInterrupt: 

In [6]:
output_metrics_csv = "t5_metrics_results.csv"
metrics_calc.evaluate_text_metrics(tokenizer, model, output_csv, output_metrics_csv)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight'

Metrics saved to 't5_metrics_results.csv'
